In [1]:
import pandas as pd
import numpy as np
import re
import math
import spacy
from tqdm import tqdm, trange
from gensim.models.phrases import Phrases, Phraser
from collections import defaultdict
import multiprocessing
from gensim.models import Word2Vec, KeyedVectors
from time import time
import matplotlib.pyplot as plt
import os
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from datasets import load_dataset
from multiprocess import Process, cpu_count
import multiprocess as mp
import spacy
from spacy import displacy

In [2]:
W2V_MODEL_PATH = "../W2V Model/"
NER_MODEL_PATH = "./model-best"

In [3]:
w2v_model = Word2Vec.load(W2V_MODEL_PATH + "w2v.model")

In [4]:
nlp_ner = spacy.load("model-best")

### Gathering Module Datasets

In [5]:
UNI_MODDESC_MAPPING = {
    "nus_dsa_mods.xlsx" : "mod_desc",
    "NTU_course_info.csv" : "Course Aims",
    "SMU_course_info.csv" : "Description",
    "SUSS_course_info.csv" : "module description",
    "SUTD_course_info.csv" : "Module description",
    "SIT_Module_Info.csv" : "Module Description "
}

UNI_MODCODE_MAPPING = {
    "nus_dsa_mods.xlsx" : "mod_code",
    "NTU_course_info.csv" : "Module Code",
    "SMU_course_info.csv" : "Module Code",
    "SUSS_course_info.csv" : "module code",
    "SUTD_course_info.csv" : "Module code",
    "SIT_Module_Info.csv" : "Module Code"
}

UNI_MODNAME_MAPPING = {
    "nus_dsa_mods.xlsx" : "mod_name",
    "NTU_course_info.csv" : "Module Name",
    "SMU_course_info.csv" : "Module Name",
    "SUSS_course_info.csv" : "module name",
    "SUTD_course_info.csv" : "Module Title",
    "SIT_Module_Info.csv" : "Module Title"
}

SKIP_ROWS = {
    "nus_dsa_mods.xlsx" : 0,
    "NTU_course_info.csv" : 0,
    "SMU_course_info.csv" : 1,
    "SUSS_course_info.csv" : 0,
    "SUTD_course_info.csv" : 5,
    "SIT_Module_Info.csv" : 0
}

MODULE_READ = "../../../Data/university_courses/"

In [6]:
modules = pd.DataFrame([], dtype='object', columns = ["school", "code", "name", "description"])
for uni, description_col in UNI_MODDESC_MAPPING.items():
    school_name = uni.split("_")[0].upper()
    print(f"Gathering module descriptions from {school_name}")
    try:
        table = pd.read_excel(MODULE_READ + uni, skiprows=SKIP_ROWS[uni])
    except:
        table = pd.read_csv(MODULE_READ + uni, skiprows=SKIP_ROWS[uni], encoding_errors='ignore')
    
    table = table[[UNI_MODCODE_MAPPING[uni], UNI_MODNAME_MAPPING[uni], UNI_MODDESC_MAPPING[uni]]].dropna().reset_index(drop=True)
    table.rename(columns = {
        UNI_MODCODE_MAPPING[uni] : "code",
        UNI_MODNAME_MAPPING[uni] : "name",
        UNI_MODDESC_MAPPING[uni] : "description"
    }, inplace=True)
    table["school"] = school_name
    
    modules = pd.concat([modules, table], axis = 0).reset_index(drop=True)
    
display(modules)

Gathering module descriptions from NUS
Gathering module descriptions from NTU
Gathering module descriptions from SMU
Gathering module descriptions from SUSS
Gathering module descriptions from SUTD
Gathering module descriptions from SIT


,school,code,name,description
0,NUS,CS1010,Programming Methodology,This module introduces the fundamental concept...
1,NUS,DSA1101,Introduction to Data Science,The abundance of data being harvested from var...
2,NUS,MA2001,Linear Algebra I,This module is a first course in linear algebr...
3,NUS,MA2002,Calculus,This is a course in single-variable calculus. ...
4,NUS,CS2040,Data Structures and Algorithms,This module introduces students to the design ...
...,...,...,...,...
173,SIT,ICT3211,Integrative Team Project,Students will be grouped into teams of 5-6 and...
174,SIT,ICT3210/ICT3110,Industry Certification Module,To keep up-to-date with the advances in techno...
175,SIT,CSC3002,Integrated Work Study Programme,Students will undertake an eight-month Integra...
176,SIT,ICT4001,Capstone Project,This is a major individual project that is to ...


### Function for preprocessing data

In [7]:
HTML_PATTERN = re.compile('<.*?>')
STOP_WORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def cleaning(chunk):
    
    # Importing libraries for parallelization later
    import re
    import nltk
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
    HTML_PATTERN = re.compile('<.*?>')
    STOP_WORDS = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    outputs = []
    for target_input in chunk:
        # convert to lower case
        target_input = target_input.lower()

        # remove websites
        target_input = re.sub(r'http\S+', ' ', target_input)

        # remove html tags
        target_input = re.sub(HTML_PATTERN, ' ', target_input)

        # remove all non-alphabets
        target_input = re.sub("[^A-Za-z']+", ' ', target_input)

        #will remove extra spaces
        target_input = re.sub(r'\s+',' ',target_input)

        # remove stopwords and lemmatize
        target_input_tokens = nltk.word_tokenize(target_input)
        target_input_tokens_wo_stopwords = [lemmatizer.lemmatize(i) for i in target_input_tokens if i not in STOP_WORDS and i]
        
        outputs.append(target_input_tokens_wo_stopwords)
    
    return outputs

### Functions for Generating Scores

In [8]:
def calc_score(cos_sim):
    """
    Calculates a score from 0 to 100 given a cosine-similarity score that ranges from -1 to 1
    
    Details:
    Let A be the angle between two vectors u and v. In other words, cos_sim = cos(A)
    score = A/180 * 100
    """
    
    # To handle computational rounding errors. Sometimes cos_sim is 1.00001 or -1.00001
    if cos_sim > 1: cos_sim = 1
    elif cos_sim < -1: cos_sim = -1
    
    return (math.pi - math.acos(cos_sim)) * 100 / math.pi

In [9]:
def get_doc2doc_score(job_desc, mod_desc, verbose = 1):
    """
    Computes the similarity score between two documents
    """
    job_desc = nlp_ner(job_desc)
    mod_desc = nlp_ner(mod_desc)
    scores = []
    OOV = [] # Stores out of vocabulary words
    
    for job_ents in job_desc.ents:
        job_ents = cleaning([job_ents.text])[0]
        for job_ent in job_ents:
            if job_ent not in w2v_model.wv: # If job_ent not found in vocabulary
                if job_ent not in OOV and job_ent not in STOP_WORDS:
                    OOV.append(job_ent)
                    if verbose:
                        print(f"JOB: {job_ent} not found in vocabulary")
                scores.append(0)
                continue
            max_cossim = -1
            best_mod_ent = None
            for mod_ents in mod_desc.ents:
                mod_ents = cleaning([mod_ents.text])[0]
                for mod_ent in mod_ents:
                    if mod_ent not in w2v_model.wv:
                        if mod_ent not in STOP_WORDS and mod_ent not in OOV:
                            OOV.append(mod_ent)
                            if verbose:
                                print(f"MODULE: {mod_ent} not found in vocabulary")
                    else:
                        cos_sim = w2v_model.wv.similarity(job_ent, mod_ent)
                        
                        # To handle computational rounding errors. Sometimes cos_sim is 1.00001 or -1.00001
                        if cos_sim > 1: cos_sim = 1
                        elif cos_sim < -1: cos_sim = -1
                            
                        if cos_sim >= max_cossim:
                            max_cossim = cos_sim
                            best_mod_ent = mod_ent
                if best_mod_ent == None:
                    if verbose:
                        print(f"No matching skills found for {job_ent} in {mod_desc}")
                    scores.append(0)
                else:
                    score = calc_score(max_cossim)
                    scores.append(score)

    return np.mean(np.array(scores))

In [10]:
def get_skill2mod_score(skill, mod_desc):
    """
    Generates a score and the skill token identified in `mod_desc` with the closest match to `skill`
    """
    mod_desc = nlp_ner(mod_desc)
    max_score = 0
    best_ent = None
    if skill not in w2v_model.wv:
        return max_score, best_ent
    for mod_ents in mod_desc.ents:
        mod_ents = cleaning([mod_ents.text])[0]
        for mod_ent in mod_ents:
            if mod_ent in w2v_model.wv:
                cos_sim = w2v_model.wv.similarity(mod_ent, skill)
                
                # To handle computational rounding errors. Sometimes cos_sim is 1.00001 or -1.00001
                if cos_sim > 1: cos_sim = 1
                elif cos_sim < -1: cos_sim = -1
                    
                score = calc_score(cos_sim)
                if max_score < score:
                    best_ent = mod_ent
                max_score = max(max_score, score)
            else:
                max_score = max(max_score, 0)
    return max_score, best_ent

In [11]:
def get_mod_recommendations(job_desc):
    """
    WARNING: This function takes about 1-2 minutes to run
    Given a job description (JD), this function will identify all skills within the JD, 
    and recommend a module per school for each identified skill
    
    Input: Job description (str)
    Output: Module recommendations, School scores (dictionary D1 of the format below, dictionary D2 of the format below)
    
    Format of D1:
    {
        JD_skill_1 : {
            school_1 : (school_1_module, score_for_school_1_module),
            school_2 : (school_2_module, score_for_school_2_module),
            school_3 : (school_3_module, score_for_school_3_module),
            ...
        },
        JD_skill_2 : {
            school_1 : (school_1_module, score_for_school_1_module),
            school_2 : (school_2_module, score_for_school_2_module),
            school_3 : (school_3_module, score_for_school_3_module),
            ...
        },
        JD_skill_3 : {
            school_1 : (school_1_module, score_for_school_1_module),
            school_2 : (school_2_module, score_for_school_2_module),
            school_3 : (school_3_module, score_for_school_3_module),
            ...
        },
        ...
    }
    
    Format of D2:
    {
        school_1 : score,
        school_2 : score,
        school_3 : score,
        ...
    }
    """
    
    all_schools = {}
    for ent in tqdm(nlp_ner(job_desc).ents):
        skill_words = cleaning([ent.text])[0]
        best_mods = {}
        for skill_word in skill_words:
            modules_copy = modules.copy()
            modules_copy['score'] = modules_copy.description.apply(lambda x: get_skill2mod_score(skill_word, x)[0])
            modules_copy = modules_copy.sort_values('score', ascending=False).drop_duplicates('school')
            for i, row in modules_copy.iterrows():
                school, code, name, description, score = row
                if school not in best_mods or best_mods[school][1]:
                    best_mods[school] = (code, score)
        all_schools[" ".join(skill_words)] = best_mods
        
    def get_school_scores(all_schools):
        """
        Assigns a score to every school. Score ranges from 0 to 100
        """
        all_scores = {}
        num_skills = len(all_schools)
        for skill in all_schools.keys():
            schools = all_schools[skill].keys()
            for school in schools:
                score = all_schools[skill][school][1]
                if school not in all_scores: all_scores[school] = 0
                all_scores[school] += score
        for school, total_score in all_scores.items():
            all_scores[school] = total_score / num_skills
        return all_scores
    
    return all_schools, get_school_scores(all_schools)

### Demo

In [12]:
job_desc = """
What a College Intern - Data Science does at HP:
Attached to the "Smart Manufacturing Application and Research Center".
Work with an enterprising team of data scientists and build solutions to track, analyze and visualize the manufacturing and outbound quality of our supplies.
Generate deep insights through the analysis of data and understanding of operational processes and turn them into actionable recommendations.
Develop methodologies for optimizing our business processes through data visualization, real-time monitoring, predictive analytics etc.
Are you a high-performer? We are looking for an individual with:
Studying Bachelor’s degree in Computer Science, Business Analytics, Information Systems, Industrial Engineering, Statistics with good experience in programming.
Excellent analytical thinking, programming (using R/Python is desirable), and problem-solving skills.
Knowledge of data analytics, data warehousing, database management (preferably using SQL) and data visualization using RShiny and Plotly.
Fundamental knowledge of statistics and probability.
Good visualization skills to create real-time dashboards and/or reports to inform trends and insights.
"""

In [13]:
mod_desc1 = """
This module covers common algorithmic techniques for solving optimisation problems, and introduces students to approaches for finding good-enough solutions to NP-hard problems. Topics covered include linear and integer programming, network flow algorithms, local search heuristics, approximation algorithms, and randomized algorithms. Through analysis and application of the techniques to a variety of canonical problems, students develop confidence to (i) appropriately model a given optimisation problem, (ii) apply appropriate algorithmic techniques to solve the problem, (iii) analyse the properties of the problem and candidate algorithms, such as time and space complexity, convergence, approximability, and optimality bound.
"""

In [14]:
mod_desc2 = """
Data visualisation is an essential tool for data analytics. This module is an introduction to data cleaning, exploration, analysis and visualisation. Students will learn how to take raw data, extract meaningful information, use statistical tools, and make visualisations. Topics include: programming in R, introduction to data storage systems, data manipulation, exploratory data analysis, dimension reduction, statistical graphics for univariate, multivariate (high-dimensional), temporal and spatial data, basic design principles and critical evaluation of visual displays of data.
"""

In [15]:
spacy.displacy.render(nlp_ner(job_desc), style = 'ent', jupyter=True)

In [16]:
spacy.displacy.render(nlp_ner(mod_desc1), style = 'ent', jupyter=True)

In [17]:
spacy.displacy.render(nlp_ner(mod_desc2), style = 'ent', jupyter=True)

In [18]:
print(f"Score: {get_skill2mod_score('visualization', mod_desc1)}")
print(f"Score: {get_skill2mod_score('visualization', mod_desc2)}")

Score: (53.63886155387202, 'search')
Score: (71.64724786243553, 'visualisation')


In [19]:
mod_reco, school_scores = get_mod_recommendations(job_desc)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [01:22<00:00,  3.61s/it]


In [20]:
mod_reco

{'analyze visualize manufacturing': {'SUTD': (50.005, 62.6140016341832),
  'NTU': ('CZ2006', 62.6140016341832),
  'SMU': ('MKTG228', 61.56321093606834),
  'SUSS': ('ANL355', 62.17314580932006),
  'SIT': ('CSC3004', 62.6140016341832),
  'NUS': ('DSA2102', 68.3238221386607)},
 'analysis data': {'SUSS': ('ANL310', 100.0),
  'NUS': ('CS1010', 100.0),
  'SUTD': (50.051, 100.0),
  'SMU': ('MKTG228', 100.0),
  'NTU': ('CZ4032', 100.0),
  'SIT': ('CSC3004', 100.0)},
 'process': {'SUTD': (50.002, 99.98900981053919),
  'SMU': ('IS424', 99.98900981053919),
  'SUSS': ('ACC353', 99.98900981053919),
  'NTU': ('CC0001', 99.98900981053919),
  'SIT': ('ICT2101', 99.98900981053919),
  'NUS': ('CS5340 ', 65.85597030390915)},
 'data visualization': {'NUS': ('CS4225', 100.0),
  'SUSS': ('MKT365', 100.0),
  'SMU': ('DSA212', 100.0),
  'NTU': ('CZ4032', 100.0),
  'SIT': ('CSC3004', 62.255993579991575),
  'SUTD': (50.017, 63.54604934317173)},
 'real time monitoring': {'NTU': ('CZ2007', 64.90214678337439),
  '

In [21]:
school_scores

{'SUTD': 79.83747017724923,
 'NTU': 88.28851334018437,
 'SMU': 88.4211271705299,
 'SUSS': 88.57483866188039,
 'SIT': 80.17783980826852,
 'NUS': 86.98658511739512}